I'm going to explore multi-dimensional inputs and outputs in this notebook.

We're going to have $y = y(y_{0}, y_{1}, ..., y_{n})$, and $x = x(x_{0}, x_{1}, ..., x_{n})$. For simplicity's sake, let $n = 1$.

I'm also going to assume each parameter of $x$ is mapped to its corresponding $y$ parameter: as in,

$x_{0} \rightarrow y_{0}$, and  $x_{1} \rightarrow y_{1}$.

There'll be functions mapping each input to its corresponding output, of course, for example,

$y_{0} = x_{0}^{2}$, and $y_{1} = x_{1}^{4}$.

I'm going to use more complex functions to strain the model a little more, but the real challenge is plotting these, because

$x = [x_{0} \; x_{1}]$, and $y = [y_{0} \; y_{1}]$.

A neural network usually has one output layer. I'm going to have to branch out the layer this time, such that each y-element has an output layer.

Also, I'm going to use a dropout probability of 0.3, since, from the dropouts notebook, that seems to be the optimal choice.

With that, I'll get started.

In [1]:
#importing libraries
import numpy as np
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
import torchbnn as bnn
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#allocating datasets and model to GPU for speed's sake
is_available = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

My data generation will be different here. I'm going to:
* Have two linspaces: x0, x1
* Have two  functions: fx0, fx1
* Have two more linspaces: y0, y1
* Form a dataframe (df) of the four linspaces
* Use train_test_split on the df

I'll think about device applications later.

I'll also keep the functions simple:

$y_{0} = f(x_{0}) = x^{5} -7x^{2} + 4$

$y_{1} = f(x_{1}) = 3x^{2} + 2x - 5$

In [3]:
#creating dataset
x0, x1 = torch.linspace(-2, 2, 1000), torch.linspace(-2, 2, 1000)

fx0 = lambda x: np.power(x, 5) - 7*np.power(x, 2) + (torch.rand(x.size())*4 + 2)
fx1 = lambda x: 3*np.power(x, 2) + 2*x - (torch.rand(x.shape)*5 + 2)

y0 = fx0(x0)
y1 = fx1(x1)

df = pd.DataFrame(
    {
        "x0":x0,
        "x1":x1,
        "y0":y0,
        "y1":y0
    }
)

#df.head()
x_train, x_test, y_train, y_test = train_test_split(df[["x0", "x1"]], df[["y0", "y1"]], test_size=0.2, random_state=1)

#reset indices and drop old indices of train and test sets
x_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

y_train.head()

,y0,y1
0,0.494648,0.494648
1,7.109888,7.109888
2,6.684902,6.684902
3,-38.015930,-38.015930
4,2.194426,2.194426


Since I'm working with multi-dimensional inputs and outputs, the model architecture has to change to accommodate the new dataset's properties.

In [4]:
class DualOutputBNN(nn.Module):
    def __init__(self, no_of_neurones, dropout_prob):
        super(DualOutputBNN, self).__init__()
        self.shared_layer = nn.Sequential(
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=2, out_features=no_of_neurones),
            nn.ReLU(),
            nn.Dropout(dropout_prob),
            bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=no_of_neurones),
        )
        self.output_layer_y0 = bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=1)
        self.output_layer_y1 = bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=no_of_neurones, out_features=1)

    def forward(self, x):
        shared = self.shared_layer(x)
        y0 = self.output_layer_y0(shared)
        y1 = self.output_layer_y1(shared)
        return y0, y1

I'll now write a function to initialise the model above, in the same way that I did in previous notebooks.

In [5]:
def initialise_model(no_of_neurones: int, dropout_prob: float, lr: float = 0.01) -> tuple:
    """
    Initialise the DualOutputBNN model with its loss functions and optimizer.

    Parameters:
    - no_of_neurones (int): Number of neurons in the hidden layer.
    - dropout_prob (float): Dropout probability.
    - lr (float): Learning rate for the optimizer. Default is 0.01.

    Returns:
    - A tuple containing the initialized model, MSE loss function, KL loss function, KL weight, and optimizer.
    """

    model = DualOutputBNN(no_of_neurones, dropout_prob).to(device)

    mse_loss = nn.MSELoss().to(device)
    kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False).to(device)
    kl_weight = 0.01

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return model, mse_loss, kl_loss, kl_weight, optimizer



Okay, I've got my dataset, split it into training and testing sets, declared my model architecture, and written the function to intialise it.

What's next is to:
* Write a function to train the model
* Write a function to test the model
* Find a way to plot the model's predictions based on testing data

At some point, I should really make a class containing all of these functions so that I don't repeat them in every notebook...

In [6]:
def train_model(model_attributes, x_train, y_train, epochs: int):
    """
    Train a Bayesian Neural Network model for a specified number of epochs.

    Parameters:
    - model_attributes: A tuple containing the model, loss functions, kl_weight, and optimizer.
    - x_train (torch.Tensor): Input tensor for the training data.
    - y_train (torch.Tensor): Target tensor for the training data.
    - epochs (int): Number of epochs to train the model.

    Returns:
    - model (torch.nn.Sequential): The trained neural network model.

    Ensures that training data and model are on the same device for efficiency.
    """

    # assert x_train.device == y_train.device == device, "Keep everything on the GPU for speed"

    model, mse_loss, kl_loss, kl_weight, optimizer = model_attributes
    model.train()

    # x_train, y_train = np.array(x_train), np.array(y_train)
    print(x_train.shape, y_train.shape)
    # x_train, y_train = torch.Tensor(x_train).to(device), torch.Tensor(y_train).to(device)

    print(f"training model...")
    for _ in range(epochs):
        y0_pred, y1_pred = model(x_train)
        y0_mse, y1_mse = mse_loss(y0_pred, y_train[:,0]), mse_loss(y1_pred, y_train[:,1])
        kl = kl_loss(model)
        y0_cost, y1_cost = y0_mse + kl_weight * kl, y1_mse + kl_weight * kl
        cost = y0_cost + y1_cost
        # cost = mse + kl_weight * kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    print(f"- Cost: {cost.item():.3f}")

    return model

Since I already produced the test set in the beginning, producing testing data in test_model() not only becomes trivial, but, with the way I produce it in the function, error-prone. I can also remove the func parameter for this reason.

The real problem is processing the produced data.

I need to:
* Predict both $y_{0}$ and $y_{1}$
* Find each output's mean_model_results and std_model_results
* Return a df: x_test, y_test, mean_model_results (for $y_{0},y_{1}$), std_model_results (for $y_{0}, y_{1}$)

So basically I'm producing a results dataframe.

In [7]:
def test_model(model, x_test, y_test):
    """
    Test a Bayesian Neural Network model to produce predictions along with mean and standard deviation.

    Parameters:
    - model (torch.nn.Sequential): The trained neural network model.
    - func (callable): The target function to compare against the model's predictions.

    Returns:
    - mean_model_results (numpy.ndarray): Array of mean predictions from the model.
    - std_model_results (numpy.ndarray): Array of standard deviations of the predictions.
    - x_test (torch.Tensor): Testing dataset inputs.
    - y_test (torch.Tensor): Testing dataset targets.

    The testing data is moved to the device specified by the global `device` variable.
    """

    #producing predictions of model of testing data, as well as mean and standard deviation of predictions
    model.eval().cpu()
    y0_pred, y1_pred = np.array([model(x_test)[0].detach().numpy() for _ in range(10000)])[:,:,0].T, np.array([model(x_test)[1].detach().numpy() for _ in range(10000)])[:,:,0].T
    print(f"produced predictions")

    mean_y0_results, std_y0_results = np.array([np.mean(y0_pred[i]) for i in range(y0_pred.shape[0])]), np.array([np.std(y0_pred[i]) for i in range(y0_pred.shape[0])])
    mean_y1_results, std_y1_results = np.array([np.mean(y1_pred[i]) for i in range(y1_pred.shape[0])]), np.array([np.std(y1_pred[i]) for i in range(y1_pred.shape[0])])

    print(f"produced means and standard deviations")
    # model_results = np.array([model(x_test) for _ in range(10000)])[:,:,0].T
    # mean_model_results = np.array([np.mean(model_results[i]) for i in range(model_results.shape[0])])
    # std_model_results = np.array([np.std(model_results[i]) for i in range(model_results.shape[0])])

    return mean_y0_results, std_y0_results, mean_y1_results, std_y1_results, x_test.cpu(), y_test.cpu() #!this function needs testing

In [8]:
x_train, x_test, y_train, y_test = np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)
x_train, x_test, y_train, y_test = torch.Tensor(x_train).to(device), torch.Tensor(x_test), torch.Tensor(y_train).to(device), torch.Tensor(y_test)

# x_train, x_test, y_train, y_test = torch.unsqueeze(torch.Tensor(x_train).to(device), dim=2), torch.unsqueeze(torch.Tensor(x_test), dim=2), torch.unsqueeze(torch.Tensor(y_train).to(device), dim=2), torch.unsqueeze(torch.Tensor(y_test), dim=2)

# print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# model_attributes = initialise_model(1000, 0.3)
# model = train_model(model_attributes, x_train, y_train, 1000)
# # model.eval()
# mean_model_results, std_model_results, x_test, y_test = test_model(model, x_test, y_test)

print(type(x_train), type(y_train), type(x_test), type(y_test))

model_attributes = initialise_model(1000, 0.3)
model = train_model(model_attributes, x_train, y_train, 1000)

mean_y0_results, std_y0_results, mean_y1_results, std_y1_results, x_test, y_test = test_model(model, x_test, y_test)

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([800, 2]) torch.Size([800, 2])


c:\Users\resul\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([800])) that is different to the input size (torch.Size([800, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


- Cost: 330.249


KeyboardInterrupt: 